In [1]:
!pip3 install ivis

     |████████████████████████████████| 645kB 7.8MB/s 
  Created wheel for annoy: filename=annoy-1.16.2-cp36-cp36m-linux_x86_64.whl size=310409 sha256=f327a52fa1388ad711a7052cf81fb519138a1d184022aafc69b5ff193564306d
  Stored in directory: /root/.cache/pip/wheels/d3/d7/68/3795670ef4c6781fc10df0d6cf83b922244aa28cd9489d1176
Successfully built annoy


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
from ivis import Ivis
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np

from keras.layers import BatchNormalization, Dense, Activation, Lambda, Input
from keras.models import Sequential, Model
from keras.callbacks import LearningRateScheduler
from keras import backend as K

from imblearn.over_sampling import SMOTE, RandomOverSampler

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.naive_bayes import GaussianNB

import itertools
import time

base_path = './drive/My Drive/'

train_data_fname = 'ant-1.5.csv'
test_data_fname = 'ant-1.6.csv'

batch_size = 128
num_classes = 2

train_df = pd.read_csv(f'{base_path}/{train_data_fname}')
test_df = pd.read_csv(f'{base_path}/{test_data_fname}')

train_df.head()

x_train = train_df.drop('bug', axis=1)
x_train.drop(['name', 'version', 'name.1'], axis=1, inplace=True)
y_train = train_df['bug']

x_test = test_df.drop('bug', axis=1)
x_test.drop(['name', 'version', 'name.1'], axis=1, inplace=True)
y_test = test_df['bug']

(y_train == 0).sum(), (y_train == 1).sum(), (y_train == 2).sum()

idx = np.where(y_train == 2)[0]
x_train.drop(idx, axis=0, inplace=True)
y_train.drop(idx, inplace=True)

x_train.shape, y_train.shape

idx = np.where(y_test > 1)[0]
x_test.drop(idx, axis=0, inplace=True)
y_test.drop(idx, inplace=True)

x_test.shape, y_test.shape

np.unique(y_train)

[(i, (y_test == i).sum()) for i in np.unique(y_test)]



"""## Helper functions"""

def parabola(x):
    return x ** 2

def get_model(units=2, n_layers=1, deep_learner=False, batchnorm=False, activation='relu'):
    """
    Returns a model.
    
    Params:
    =======
    units: int - Number of input units. Only for deep learners.
    layers: int - Number of hidden layers. Only for deep learners.
    deep_learner: boolean - To use deep learner, set to True.
    batchnorm: boolean - If True, uses Batch Normalization.
    
    Returns:
    ========
    An instance of keras.Sequential or AdaBoost from sklearn.
    """
    if deep_learner:
        layers = []
        
        # Add first hidden layer set.
        first = Dense(units, input_shape=(units,))
        layers.append(first)
        
        if batchnorm:
            layers.append(BatchNormalization())
        
        layers.append(Activation(activation))
        
        # Add the other hidden layers.
        for i in range(n_layers - 1):
            layers.append(Dense(units))
            
            if batchnorm:
                layers.append(BatchNormalization())
            
            layers.append(Activation(activation))
        
        # Add the output layer.
        layers.append(Dense(1))
        layers.append(Activation('sigmoid'))
        
        return Sequential(layers)
    else:
        return AdaBoostClassifier()

def get_oversampler(name=None):
    """
    Returns an oversampler.
    
    Params:
    =======
    name: str or None - One of None, 'smote', and 'random'.
    """
    if name is None:
        return None
    if name == 'smote':
        return SMOTE()
    if name == 'random':
        return RandomOverSampler()

def run_one_experiment(x_resampled, y_resampled, x_test, y_test, deep_learner=False, n_layers=1, batchnorm=False, lipschitz_lr=False, 
                       reduce_data=False, embedding_dims=2, k=7, oversample=False, oversampler=None,
                       activation='relu'):
    """
    Runs one experiment.
    
    Params:
    =======
    x, y: np.array - Input data
    deep_learner: boolean - If True, uses a deep learner.
    layers: int - Number of hidden layers. Deep learners only.
    batchnorm: boolean - If True, uses Batch Normalization in deep learner.
    lipschitz_lr: boolean - If True, uses LipschitzLR
    reduce_data: boolean - If True, reduces data using Ivis.
    k: int - k used in Ivis reduction.
    embedding_dims: int - Number of dimensions to reduce data to
    oversample: boolean - If True, oversamples dataset.
    oversampler: str or None - One of None, 'smote', and 'random'.
    
    Returns:
    ========
    prec, recall, f1, time - Precision, recall, and F-1 scores of the model, along with runtime
    """    
    
    # Step 3: Get the model
    model = get_model(units=x_resampled.shape[1],n_layers=n_layers, 
                      deep_learner=deep_learner, batchnorm=batchnorm,
                      activation=activation)
    
    # Step 4: Train model
    if deep_learner:
        ####################
        # LipschitzLR code #
        ####################    
        def lr_schedule(epoch):
            """Learning Rate Schedule
            # Arguments
                epoch (int): The number of epochs
            # Returns
                lr (float32): learning rate
            """

            Kz = 0.
            for i in range((len(x_resampled) - 1) // batch_size + 1):
                start_i = i * batch_size
                end_i = start_i + batch_size
                xb = x_resampled[start_i:end_i]

                activ = np.linalg.norm(func([xb]))
                if activ > Kz:
                    Kz = activ

            K_ = ((num_classes - 1) * Kz) / (num_classes * batch_size)
            lr = 1 / K_
            #print('Epoch', epoch + 1, 'LR =', lr)
            return lr
        ####################
        
        if lipschitz_lr:
            model.compile(loss='binary_crossentropy',
                  optimizer='sgd')
            func = K.function([model.layers[0].input], [model.layers[-2].output])
            lr_scheduler = LearningRateScheduler(lr_schedule)
            
            start = time.time()
            model.fit(x_resampled, y_resampled, batch_size=batch_size, verbose=0, 
                      validation_data=(x_test, y_test), epochs=100, callbacks=[lr_scheduler])
            end = time.time()
        else:
            model.compile(loss='binary_crossentropy',
                  optimizer='adam')
            
            start = time.time()
            model.fit(x_resampled, y_resampled, batch_size=batch_size, verbose=0, 
                      validation_data=(x_test, y_test), epochs=100)
            end = time.time()
        
    else:
        start = time.time()
        model.fit(x_resampled, y_resampled)
        end = time.time()
    
    # Step 5: Evaluate model
    evaluators = [precision_score, recall_score, f1_score]
    if deep_learner:
        prec, recall, f1 = [f(y_test, model.predict_classes(x_test)) for f in evaluators]
    else:
        prec, recall, f1 = [f(y_test, model.predict(x_test)) for f in evaluators]
    
    return prec, recall, f1, (end - start)

def run_20_experiments(reduce_data=False, oversample=False, oversampler=None, 
                       embedding_dims=2, k=7, lipschitz_lr=False, activation='relu', **kwargs):
    global x_train, y_train, x_test, y_test
    prec = []
    recall = []
    f1 = []
    times = []
    
    count = 0

    # Step 1: Reduce dimensions --> (x_*_reduced, y_*)
    x_train_reduced = np.array(x_train)
    x_test_reduced = np.array(x_test)
    y_train = np.array(y_train)
    y_test = np.array(y_test)

    if reduce_data:
        ivis = Ivis(embedding_dims=embedding_dims, k=k, verbose=0)
        x_train_reduced = ivis.fit_transform(x_train_reduced)
        x_test_reduced = ivis.transform(x_test_reduced)

    # Step 2: Oversample data --> (x_resampled, y_resampled)
    x_resampled, y_resampled = x_train_reduced, y_train
    if oversample:
        sampler = get_oversampler(name=oversampler)
        x_resampled, y_resampled = sampler.fit_resample(x_train_reduced, y_train)

    while count != 20:
        try:          
            precision, rec, f1_, time_ = run_one_experiment(x_resampled, y_resampled,
                                                            x_test_reduced, y_test, 
                                                            lipschitz_lr=lipschitz_lr, 
                                                            activation=activation, **kwargs)
            prec.append(precision)
            recall.append(rec)
            f1.append(f1_)
            times.append(time_)
            count += 1
        except KeyboardInterrupt:
            raise
        except:
            print('Retrying...')
            if lipschitz_lr:
                pass
            else:
                raise
    
    print('Precision:', prec, '\nRecall:', recall, '\nF1:', f1, '\nRuntime:', times)

"""## Run recommended experiments"""

def get_experimental_config():
    dl_options = [False, True]
    layers_options = [1, 2, 3]
    batchnorm_options = [True]  # Reduces #experiments
    lipschitz_options = [False, True]
    reduce_options = [False, True]
    embedding_options = [2, 3]
    k_options = [3, 4, 5, 6, 7]
    oversample_options = [False, True]
    oversampler_options = ['random', None]  # Not using smote
    activation_options = ['relu', parabola]

    for (dl, layers, bn, lipschitz, reduce, emb, k, over, sampler, activation) in itertools.product(dl_options,
                                                                                       layers_options,
                                                                                       batchnorm_options,
                                                                                       lipschitz_options,
                                                                                       reduce_options,
                                                                                       embedding_options,
                                                                                       k_options,
                                                                                       oversample_options,
                                                                                       oversampler_options,
                                                                                       activation_options):
        # Constraints:
        if dl == False:
            if layers != 1 or lipschitz != False or activation != 'relu' or bn != True:
                continue
        if not reduce:
            if emb != 2 or k != 3:
                continue
        if not over:
            if sampler is not None:
                continue
        else:
            if sampler is None:
                continue

        # Skip over completed configs
        complete = [
                {'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': False, 'embedding_dims': 2, 'k': 3, 'oversample': True, 'oversampler': 'random'},
                {'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': False, 'embedding_dims': 2, 'k': 3, 'oversample': True, 'oversampler': 'random'},
                {'deep_learner': True, 'n_layers': 3, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': False, 'embedding_dims': 2, 'k': 3, 'oversample': True, 'oversampler': 'random'},
                {'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 5, 'oversample': True, 'oversampler': 'random'},
                {'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 7, 'oversample': True, 'oversampler': 'random'},
                {'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 7, 'oversample': True, 'oversampler': 'random'},
                {'deep_learner': False, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 6, 'oversample': True, 'oversampler': 'random'},
                {'deep_learner': False, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 6, 'oversample': True, 'oversampler': 'random'},
                {'deep_learner': False, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': False, 'embedding_dims': 2, 'k': 3, 'oversample': False, 'oversampler': None, 'activation': 'relu'},
                {'deep_learner': False, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': False, 'embedding_dims': 2, 'k': 3, 'oversample': True, 'oversampler': 'random', 'activation': 'relu'},
                {'deep_learner': False, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 3, 'oversample': False, 'oversampler': None, 'activation': 'relu'},
                {'deep_learner': False, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 3, 'oversample': True, 'oversampler': 'random', 'activation': 'relu'},
                {'deep_learner': False, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 4, 'oversample': False, 'oversampler': None, 'activation': 'relu'},
                {'deep_learner': False, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 4, 'oversample': True, 'oversampler': 'random', 'activation': 'relu'},
                {'deep_learner': False, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 5, 'oversample': False, 'oversampler': None, 'activation': 'relu'},
                {'deep_learner': False, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 5, 'oversample': True, 'oversampler': 'random', 'activation': 'relu'},
                {'deep_learner': False, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 6, 'oversample': False, 'oversampler': None, 'activation': 'relu'},
                {'deep_learner': False, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 6, 'oversample': True, 'oversampler': 'random', 'activation': 'relu'},
                {'deep_learner': False, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 7, 'oversample': False, 'oversampler': None, 'activation': 'relu'},
                {'deep_learner': False, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 7, 'oversample': True, 'oversampler': 'random', 'activation': 'relu'},
                {'deep_learner': False, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 3, 'oversample': False, 'oversampler': None, 'activation': 'relu'},
                {'deep_learner': False, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 3, 'oversample': True, 'oversampler': 'random', 'activation': 'relu'},
                {'deep_learner': False, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 4, 'oversample': False, 'oversampler': None, 'activation': 'relu'},
                {'deep_learner': False, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 4, 'oversample': True, 'oversampler': 'random', 'activation': 'relu'},
                {'deep_learner': False, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 5, 'oversample': False, 'oversampler': None, 'activation': 'relu'},
                {'deep_learner': False, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 5, 'oversample': True, 'oversampler': 'random', 'activation': 'relu'},
                {'deep_learner': False, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 6, 'oversample': False, 'oversampler': None, 'activation': 'relu'},
                {'deep_learner': False, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 6, 'oversample': True, 'oversampler': 'random', 'activation': 'relu'},
                {'deep_learner': False, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 7, 'oversample': False, 'oversampler': None, 'activation': 'relu'},
                {'deep_learner': False, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 7, 'oversample': True, 'oversampler': 'random', 'activation': 'relu'},
                {'deep_learner': False, 'n_layers': 1, 'batchnorm': False, 'lipschitz_lr': False, 'reduce_data': False, 'embedding_dims': 2, 'k': 3, 'oversample': False, 'oversampler': None, 'activation': 'relu'},
                {'deep_learner': False, 'n_layers': 1, 'batchnorm': False, 'lipschitz_lr': False, 'reduce_data': False, 'embedding_dims': 2, 'k': 3, 'oversample': True, 'oversampler': 'random', 'activation': 'relu'},
                {'deep_learner': False, 'n_layers': 1, 'batchnorm': False, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 3, 'oversample': False, 'oversampler': None, 'activation': 'relu'},
                {'deep_learner': False, 'n_layers': 1, 'batchnorm': False, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 3, 'oversample': True, 'oversampler': 'random', 'activation': 'relu'},
                {'deep_learner': False, 'n_layers': 1, 'batchnorm': False, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 4, 'oversample': False, 'oversampler': None, 'activation': 'relu'},
                {'deep_learner': False, 'n_layers': 1, 'batchnorm': False, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 4, 'oversample': True, 'oversampler': 'random', 'activation': 'relu'},
                {'deep_learner': False, 'n_layers': 1, 'batchnorm': False, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 5, 'oversample': False, 'oversampler': None, 'activation': 'relu'},
                {'deep_learner': False, 'n_layers': 1, 'batchnorm': False, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 5, 'oversample': True, 'oversampler': 'random', 'activation': 'relu'},
                {'deep_learner': False, 'n_layers': 1, 'batchnorm': False, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 6, 'oversample': False, 'oversampler': None, 'activation': 'relu'},
                {'deep_learner': False, 'n_layers': 1, 'batchnorm': False, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 6, 'oversample': True, 'oversampler': 'random', 'activation': 'relu'},
                {'deep_learner': False, 'n_layers': 1, 'batchnorm': False, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 7, 'oversample': False, 'oversampler': None, 'activation': 'relu'},
                {'deep_learner': False, 'n_layers': 1, 'batchnorm': False, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 7, 'oversample': True, 'oversampler': 'random', 'activation': 'relu'},
                {'deep_learner': False, 'n_layers': 1, 'batchnorm': False, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 3, 'oversample': False, 'oversampler': None, 'activation': 'relu'},
                {'deep_learner': False, 'n_layers': 1, 'batchnorm': False, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 3, 'oversample': True, 'oversampler': 'random', 'activation': 'relu'},
                {'deep_learner': False, 'n_layers': 1, 'batchnorm': False, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 4, 'oversample': False, 'oversampler': None, 'activation': 'relu'},
                {'deep_learner': False, 'n_layers': 1, 'batchnorm': False, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 4, 'oversample': True, 'oversampler': 'random', 'activation': 'relu'},
                {'deep_learner': False, 'n_layers': 1, 'batchnorm': False, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 5, 'oversample': False, 'oversampler': None, 'activation': 'relu'},
                {'deep_learner': False, 'n_layers': 1, 'batchnorm': False, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 5, 'oversample': True, 'oversampler': 'random', 'activation': 'relu'},
                {'deep_learner': False, 'n_layers': 1, 'batchnorm': False, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 6, 'oversample': False, 'oversampler': None, 'activation': 'relu'},
                {'deep_learner': False, 'n_layers': 1, 'batchnorm': False, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 6, 'oversample': True, 'oversampler': 'random', 'activation': 'relu'},
                {'deep_learner': False, 'n_layers': 1, 'batchnorm': False, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 7, 'oversample': False, 'oversampler': None, 'activation': 'relu'},
                {'deep_learner': False, 'n_layers': 1, 'batchnorm': False, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 7, 'oversample': True, 'oversampler': 'random', 'activation': 'relu'},
                {'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': False, 'embedding_dims': 2, 'k': 3, 'oversample': False, 'oversampler': None, 'activation': 'relu'},
                {'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': False, 'embedding_dims': 2, 'k': 3, 'oversample': False, 'oversampler': None, 'activation': parabola},
                {'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': False, 'embedding_dims': 2, 'k': 3, 'oversample': True, 'oversampler': 'random', 'activation': 'relu'},
                {'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': False, 'embedding_dims': 2, 'k': 3, 'oversample': True, 'oversampler': 'random', 'activation': parabola},
                {'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 3, 'oversample': False, 'oversampler': None, 'activation': 'relu'},
                {"deep_learner": True, "n_layers": 1, "batchnorm": True, "lipschitz_lr": False, "reduce_data": True, "embedding_dims": 2, "k": 3, "oversample": False, "oversampler": None, "activation": parabola},
{"deep_learner": True, "n_layers": 1, "batchnorm": True, "lipschitz_lr": False, "reduce_data": True, "embedding_dims": 2, "k": 3, "oversample": True, "oversampler": "random", "activation": "relu"},
{"deep_learner": True, "n_layers": 1, "batchnorm": True, "lipschitz_lr": False, "reduce_data": True, "embedding_dims": 2, "k": 3, "oversample": True, "oversampler": "random", "activation": parabola},
{"deep_learner": True, "n_layers": 1, "batchnorm": True, "lipschitz_lr": False, "reduce_data": True, "embedding_dims": 2, "k": 4, "oversample": False, "oversampler": None, "activation": "relu"},
{"deep_learner": True, "n_layers": 1, "batchnorm": True, "lipschitz_lr": False, "reduce_data": True, "embedding_dims": 2, "k": 4, "oversample": False, "oversampler": None, "activation": parabola},
{"deep_learner": True, "n_layers": 1, "batchnorm": True, "lipschitz_lr": False, "reduce_data": True, "embedding_dims": 2, "k": 4, "oversample": True, "oversampler": "random", "activation": "relu"},
{"deep_learner": True, "n_layers": 1, "batchnorm": True, "lipschitz_lr": False, "reduce_data": True, "embedding_dims": 2, "k": 4, "oversample": True, "oversampler": "random", "activation": parabola},
{"deep_learner": True, "n_layers": 1, "batchnorm": True, "lipschitz_lr": False, "reduce_data": True, "embedding_dims": 2, "k": 5, "oversample": False, "oversampler": None, "activation": "relu"},
{"deep_learner": True, "n_layers": 1, "batchnorm": True, "lipschitz_lr": False, "reduce_data": True, "embedding_dims": 2, "k": 5, "oversample": False, "oversampler": None, "activation": parabola},
{"deep_learner": True, "n_layers": 1, "batchnorm": True, "lipschitz_lr": False, "reduce_data": True, "embedding_dims": 2, "k": 5, "oversample": True, "oversampler": "random", "activation": "relu"},
{"deep_learner": True, "n_layers": 1, "batchnorm": True, "lipschitz_lr": False, "reduce_data": True, "embedding_dims": 2, "k": 5, "oversample": True, "oversampler": "random", "activation": parabola},
{'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 6, 'oversample': False, 'oversampler': None, 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 6, 'oversample': False, 'oversampler': None, 'activation': parabola},
{'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 6, 'oversample': True, 'oversampler': 'random', 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 6, 'oversample': True, 'oversampler': 'random', 'activation': parabola},
{'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 7, 'oversample': False, 'oversampler': None, 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 7, 'oversample': False, 'oversampler': None, 'activation': parabola},
{'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 7, 'oversample': True, 'oversampler': 'random', 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 7, 'oversample': True, 'oversampler': 'random', 'activation': parabola},
{'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 3, 'oversample': False, 'oversampler': None, 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 3, 'oversample': False, 'oversampler': None, 'activation': parabola},
{'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 3, 'oversample': True, 'oversampler': 'random', 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 3, 'oversample': True, 'oversampler': 'random', 'activation': parabola},
{'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 4, 'oversample': False, 'oversampler': None, 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 4, 'oversample': False, 'oversampler': None, 'activation': parabola},
{'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 4, 'oversample': True, 'oversampler': 'random', 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 4, 'oversample': True, 'oversampler': 'random', 'activation': parabola},
{'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 5, 'oversample': False, 'oversampler': None, 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 5, 'oversample': False, 'oversampler': None, 'activation': parabola},
{'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 5, 'oversample': True, 'oversampler': 'random', 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 5, 'oversample': True, 'oversampler': 'random', 'activation': parabola},
{'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 6, 'oversample': False, 'oversampler': None, 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 6, 'oversample': False, 'oversampler': None, 'activation': parabola},
{'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 6, 'oversample': True, 'oversampler': 'random', 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 6, 'oversample': True, 'oversampler': 'random', 'activation': parabola},
{'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 7, 'oversample': False, 'oversampler': None, 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 7, 'oversample': False, 'oversampler': None, 'activation': parabola},
{'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 7, 'oversample': True, 'oversampler': 'random', 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 7, 'oversample': True, 'oversampler': 'random', 'activation': parabola},
{'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': False, 'embedding_dims': 2, 'k': 3, 'oversample': False, 'oversampler': None, 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': False, 'embedding_dims': 2, 'k': 3, 'oversample': False, 'oversampler': None, 'activation': parabola},
{'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': False, 'embedding_dims': 2, 'k': 3, 'oversample': True, 'oversampler': 'random', 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': False, 'embedding_dims': 2, 'k': 3, 'oversample': True, 'oversampler': 'random', 'activation': parabola},
{'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 2, 'k': 3, 'oversample': False, 'oversampler': None, 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 2, 'k': 3, 'oversample': False, 'oversampler': None, 'activation': parabola},
{'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 2, 'k': 3, 'oversample': True, 'oversampler': 'random', 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 2, 'k': 3, 'oversample': True, 'oversampler': 'random', 'activation': parabola},
{'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 2, 'k': 4, 'oversample': False, 'oversampler': None, 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 2, 'k': 4, 'oversample': False, 'oversampler': None, 'activation': parabola},
{'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 2, 'k': 4, 'oversample': True, 'oversampler': 'random', 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 2, 'k': 4, 'oversample': True, 'oversampler': 'random', 'activation': parabola},
{'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 2, 'k': 5, 'oversample': False, 'oversampler': None, 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 2, 'k': 5, 'oversample': False, 'oversampler': None, 'activation': parabola},
{'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 2, 'k': 5, 'oversample': True, 'oversampler': 'random', 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 2, 'k': 5, 'oversample': True, 'oversampler': 'random', 'activation': parabola},
{'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 2, 'k': 6, 'oversample': False, 'oversampler': None, 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 2, 'k': 6, 'oversample': False, 'oversampler': None, 'activation': parabola},
{'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 2, 'k': 6, 'oversample': True, 'oversampler': 'random', 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 2, 'k': 6, 'oversample': True, 'oversampler': 'random', 'activation': parabola},
{'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 2, 'k': 7, 'oversample': False, 'oversampler': None, 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 2, 'k': 7, 'oversample': False, 'oversampler': None, 'activation': parabola},
{'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 2, 'k': 7, 'oversample': True, 'oversampler': 'random', 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 2, 'k': 7, 'oversample': True, 'oversampler': 'random', 'activation': parabola},
{'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 3, 'k': 3, 'oversample': False, 'oversampler': None, 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 3, 'k': 3, 'oversample': False, 'oversampler': None, 'activation': parabola},
{'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 3, 'k': 3, 'oversample': True, 'oversampler': 'random', 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 3, 'k': 3, 'oversample': True, 'oversampler': 'random', 'activation': parabola},
{'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 3, 'k': 4, 'oversample': False, 'oversampler': None, 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 3, 'k': 4, 'oversample': False, 'oversampler': None, 'activation': parabola},
{'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 3, 'k': 4, 'oversample': True, 'oversampler': 'random', 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 3, 'k': 4, 'oversample': True, 'oversampler': 'random', 'activation': parabola},
{'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 3, 'k': 5, 'oversample': False, 'oversampler': None, 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 3, 'k': 5, 'oversample': False, 'oversampler': None, 'activation': parabola},
{'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 3, 'k': 5, 'oversample': True, 'oversampler': 'random', 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 3, 'k': 5, 'oversample': True, 'oversampler': 'random', 'activation': parabola},
{'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 3, 'k': 6, 'oversample': False, 'oversampler': None, 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 3, 'k': 6, 'oversample': False, 'oversampler': None, 'activation': parabola},
{'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 3, 'k': 6, 'oversample': True, 'oversampler': 'random', 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 3, 'k': 6, 'oversample': True, 'oversampler': 'random', 'activation': parabola},
{'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 3, 'k': 7, 'oversample': False, 'oversampler': None, 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 3, 'k': 7, 'oversample': False, 'oversampler': None, 'activation': parabola},
{'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 3, 'k': 7, 'oversample': True, 'oversampler': 'random', 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 3, 'k': 7, 'oversample': True, 'oversampler': 'random', 'activation': parabola},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': False, 'embedding_dims': 2, 'k': 3, 'oversample': False, 'oversampler': None, 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': False, 'embedding_dims': 2, 'k': 3, 'oversample': False, 'oversampler': None, 'activation': parabola},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': False, 'embedding_dims': 2, 'k': 3, 'oversample': True, 'oversampler': 'random', 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': False, 'embedding_dims': 2, 'k': 3, 'oversample': True, 'oversampler': 'random', 'activation': parabola},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 3, 'oversample': False, 'oversampler': None, 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 3, 'oversample': False, 'oversampler': None, 'activation': parabola},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 3, 'oversample': True, 'oversampler': 'random', 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 3, 'oversample': True, 'oversampler': 'random', 'activation': parabola},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 4, 'oversample': False, 'oversampler': None, 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 4, 'oversample': False, 'oversampler': None, 'activation': parabola},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 4, 'oversample': True, 'oversampler': 'random', 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 4, 'oversample': True, 'oversampler': 'random', 'activation': parabola},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 5, 'oversample': False, 'oversampler': None, 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 5, 'oversample': False, 'oversampler': None, 'activation': parabola},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 5, 'oversample': True, 'oversampler': 'random', 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 5, 'oversample': True, 'oversampler': 'random', 'activation': parabola},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 6, 'oversample': False, 'oversampler': None, 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 6, 'oversample': False, 'oversampler': None, 'activation': parabola},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 6, 'oversample': True, 'oversampler': 'random', 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 6, 'oversample': True, 'oversampler': 'random', 'activation': parabola},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 7, 'oversample': False, 'oversampler': None, 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 7, 'oversample': False, 'oversampler': None, 'activation': parabola},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 7, 'oversample': True, 'oversampler': 'random', 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 7, 'oversample': True, 'oversampler': 'random', 'activation': parabola},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 3, 'oversample': False, 'oversampler': None, 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 3, 'oversample': False, 'oversampler': None, 'activation': parabola},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 3, 'oversample': True, 'oversampler': 'random', 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 3, 'oversample': True, 'oversampler': 'random', 'activation': parabola},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 4, 'oversample': False, 'oversampler': None, 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 4, 'oversample': False, 'oversampler': None, 'activation': parabola},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 4, 'oversample': True, 'oversampler': 'random', 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 4, 'oversample': True, 'oversampler': 'random', 'activation': parabola},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 5, 'oversample': False, 'oversampler': None, 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 5, 'oversample': False, 'oversampler': None, 'activation': parabola},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 5, 'oversample': True, 'oversampler': 'random', 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 5, 'oversample': True, 'oversampler': 'random', 'activation': parabola},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 6, 'oversample': False, 'oversampler': None, 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 6, 'oversample': False, 'oversampler': None, 'activation': parabola},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 6, 'oversample': True, 'oversampler': 'random', 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 6, 'oversample': True, 'oversampler': 'random', 'activation': parabola},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 7, 'oversample': False, 'oversampler': None, 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 7, 'oversample': False, 'oversampler': None, 'activation': parabola},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 7, 'oversample': True, 'oversampler': 'random', 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 7, 'oversample': True, 'oversampler': 'random', 'activation': parabola},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': False, 'embedding_dims': 2, 'k': 3, 'oversample': False, 'oversampler': None, 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': False, 'embedding_dims': 2, 'k': 3, 'oversample': False, 'oversampler': None, 'activation': parabola},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': False, 'embedding_dims': 2, 'k': 3, 'oversample': True, 'oversampler': 'random', 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': False, 'embedding_dims': 2, 'k': 3, 'oversample': True, 'oversampler': 'random', 'activation': parabola},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 2, 'k': 3, 'oversample': False, 'oversampler': None, 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 2, 'k': 3, 'oversample': False, 'oversampler': None, 'activation': parabola},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 2, 'k': 3, 'oversample': True, 'oversampler': 'random', 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 2, 'k': 3, 'oversample': True, 'oversampler': 'random', 'activation': parabola},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 2, 'k': 4, 'oversample': False, 'oversampler': None, 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 2, 'k': 4, 'oversample': False, 'oversampler': None, 'activation': parabola},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 2, 'k': 4, 'oversample': True, 'oversampler': 'random', 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 2, 'k': 4, 'oversample': True, 'oversampler': 'random', 'activation': parabola},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 2, 'k': 5, 'oversample': False, 'oversampler': None, 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 2, 'k': 5, 'oversample': False, 'oversampler': None, 'activation': parabola},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 2, 'k': 5, 'oversample': True, 'oversampler': 'random', 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 2, 'k': 5, 'oversample': True, 'oversampler': 'random', 'activation': parabola},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 2, 'k': 6, 'oversample': False, 'oversampler': None, 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 2, 'k': 6, 'oversample': False, 'oversampler': None, 'activation': parabola},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 2, 'k': 6, 'oversample': True, 'oversampler': 'random', 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 2, 'k': 6, 'oversample': True, 'oversampler': 'random', 'activation': parabola},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 2, 'k': 7, 'oversample': False, 'oversampler': None, 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 2, 'k': 7, 'oversample': False, 'oversampler': None, 'activation': parabola},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 2, 'k': 7, 'oversample': True, 'oversampler': 'random', 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 2, 'k': 7, 'oversample': True, 'oversampler': 'random', 'activation': parabola},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 3, 'k': 3, 'oversample': False, 'oversampler': None, 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 3, 'k': 3, 'oversample': False, 'oversampler': None, 'activation': parabola},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 3, 'k': 3, 'oversample': True, 'oversampler': 'random', 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 3, 'k': 3, 'oversample': True, 'oversampler': 'random', 'activation': parabola},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 3, 'k': 4, 'oversample': False, 'oversampler': None, 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 3, 'k': 4, 'oversample': False, 'oversampler': None, 'activation': parabola},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 3, 'k': 4, 'oversample': True, 'oversampler': 'random', 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 3, 'k': 4, 'oversample': True, 'oversampler': 'random', 'activation': parabola},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 3, 'k': 5, 'oversample': False, 'oversampler': None, 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 3, 'k': 5, 'oversample': False, 'oversampler': None, 'activation': parabola},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 3, 'k': 5, 'oversample': True, 'oversampler': 'random', 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 3, 'k': 5, 'oversample': True, 'oversampler': 'random', 'activation': parabola},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 3, 'k': 6, 'oversample': False, 'oversampler': None, 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 3, 'k': 6, 'oversample': False, 'oversampler': None, 'activation': parabola},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 3, 'k': 6, 'oversample': True, 'oversampler': 'random', 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 3, 'k': 6, 'oversample': True, 'oversampler': 'random', 'activation': parabola},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 3, 'k': 7, 'oversample': False, 'oversampler': None, 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 3, 'k': 7, 'oversample': False, 'oversampler': None, 'activation': parabola},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 3, 'k': 7, 'oversample': True, 'oversampler': 'random', 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 3, 'k': 7, 'oversample': True, 'oversampler': 'random', 'activation': parabola},
{'deep_learner': True, 'n_layers': 3, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': False, 'embedding_dims': 2, 'k': 3, 'oversample': False, 'oversampler': None, 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 3, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': False, 'embedding_dims': 2, 'k': 3, 'oversample': False, 'oversampler': None, 'activation': parabola},
{'deep_learner': True, 'n_layers': 3, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': False, 'embedding_dims': 2, 'k': 3, 'oversample': True, 'oversampler': 'random', 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 3, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': False, 'embedding_dims': 2, 'k': 3, 'oversample': True, 'oversampler': 'random', 'activation': parabola},
{'deep_learner': True, 'n_layers': 3, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 3, 'oversample': False, 'oversampler': None, 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 3, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 3, 'oversample': False, 'oversampler': None, 'activation': parabola},
{'deep_learner': True, 'n_layers': 3, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 3, 'oversample': True, 'oversampler': 'random', 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 3, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 3, 'oversample': True, 'oversampler': 'random', 'activation': parabola},
{'deep_learner': True, 'n_layers': 3, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 4, 'oversample': False, 'oversampler': None, 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 3, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 4, 'oversample': False, 'oversampler': None, 'activation': parabola},
{'deep_learner': True, 'n_layers': 3, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 4, 'oversample': True, 'oversampler': 'random', 'activation': 'relu'},
{'deep_learner': True, 'n_layers': 3, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 4, 'oversample': True, 'oversampler': 'random', 'activation': parabola},
{'deep_learner': True, 'n_layers': 3, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 5, 'oversample': False, 'oversampler': None, 'activation': 'relu'}
        ]

        config = {
            'deep_learner': dl,
            'n_layers': layers,
            'batchnorm': bn,
            'lipschitz_lr': lipschitz,
            'reduce_data': reduce,
            'embedding_dims': emb,
            'k': k,
            'oversample': over,
            'oversampler': sampler,
            'activation': activation
        }
        if config in complete:
            continue

        yield config


Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [4]:
count = 0
for _ in get_experimental_config():
    count += 1
print(count * 20)

1500


In [0]:
for i, config in enumerate(get_experimental_config()):
    print('Experiment', i + 1, '\b:', config)
    run_20_experiments(**config)
    print()

Experiment 1 : {'deep_learner': True, 'n_layers': 3, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 5, 'oversample': False, 'oversampler': None, 'activation': <function parabola at 0x7f70d2772510>}
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
















/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: P

Precision: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] 
Recall: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] 
F1: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] 
Runtime: [3.973482131958008, 4.50599479675293, 4.537250995635986, 4.821887493133545, 5.483065366744995, 5.4946982860565186, 5.958341836929321, 6.1966328620910645, 6.905360698699951, 6.570889234542847, 7.075550556182861, 7.015294075012207, 7.626479148864746, 7.877249240875244, 8.659815549850464, 8.670096397399902, 8.518027305603027, 8.80346131324768, 9.214348077774048, 9.536783695220947]

Experiment 2 : {'deep_learner': True, 'n_layers': 3, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 5, 'oversample': True, 'oversampler': 'random', 'activation': 'relu'}


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Precision: [0.24516129032258063, 0.3191489361702128, 0.3448275862068966, 0.18099547511312217, 0.3037974683544304, 0.24375, 0.28169014084507044, 0.2019704433497537, 0.3157894736842105, 0.22727272727272727, 0.20833333333333334, 0.21897810218978103, 0.17587939698492464, 0.2535211267605634, 0.22123893805309736, 0.4, 0.30357142857142855, 0.23214285714285715, 0.25925925925925924, 0.3125] 
Recall: [0.8260869565217391, 0.6521739130434783, 0.43478260869565216, 0.8695652173913043, 0.5217391304347826, 0.8478260869565217, 0.43478260869565216, 0.8913043478260869, 0.2608695652173913, 0.8695652173913043, 0.8695652173913043, 0.6521739130434783, 0.7608695652173914, 0.782608695652174, 0.5434782608695652, 0.2608695652173913, 0.7391304347826086, 0.8478260869565217, 0.7608695652173914, 0.43478260869565216] 
F1: [0.37810945273631835, 0.4285714285714286, 0.38461538461538464, 0.299625468164794, 0.384, 0.3786407766990291, 0.3418803418803419, 0.3293172690763052, 0.28571428571428575, 0.3603603603603604, 0.336134

...

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F

Precision: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3333333333333333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.25, 0.0, 0.09090909090909091] 
Recall: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.06521739130434782, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.021739130434782608, 0.0, 0.021739130434782608] 
F1: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.10909090909090909, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.04, 0.0, 0.03508771929824561] 
Runtime: [113.40494132041931, 113.78584051132202, 113.1633882522583, 113.82185697555542, 114.20353198051453, 114.46936798095703, 114.57763767242432, 115.2271499633789, 115.06416988372803, 115.75492405891418, 115.41520047187805, 117.15929460525513, 119.61149597167969, 122.50151228904724, 131.67003726959229, 126.6408896446228, 118.85140037536621, 119.14644360542297, 119.1699869632721, 119.49128675460815]

Experiment 18 : {'deep_learner': True, 'n_layers': 3, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 4, 'o

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: P

Precision: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2, 0.0, 0.0, 0.0, 0.2727272727272727, 0.07142857142857142, 0.3333333333333333, 0.18181818181818182, 0.0, 0.09523809523809523, 0.0, 0.0, 0.0, 0.3157894736842105] 
Recall: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.043478260869565216, 0.0, 0.0, 0.0, 0.06521739130434782, 0.021739130434782608, 0.10869565217391304, 0.043478260869565216, 0.0, 0.08695652173913043, 0.0, 0.0, 0.0, 0.13043478260869565] 
F1: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.07142857142857142, 0.0, 0.0, 0.0, 0.10526315789473684, 0.03333333333333333, 0.16393442622950818, 0.07017543859649122, 0.0, 0.0909090909090909, 0.0, 0.0, 0.0, 0.1846153846153846] 
Runtime: [132.16067171096802, 132.71704149246216, 133.00823736190796, 133.4142484664917, 133.677392244339, 134.0519027709961, 134.25174975395203, 134.9228036403656, 135.07135891914368, 135.51892352104187, 135.60038495063782, 136.19860005378723, 136.21152186393738, 137.0704984664917, 136.86263346672058, 137.6948800086975, 137.43612337112427, 138.598131

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: P